## Based on Arda instructions after I tried to do clustering though Kmeans samples

Arda: So I think you should not do Kmeans clustering, 1) the umap space is not a good space to do any analysis on just for visualization. For the gene expression space, K-means will not be optimal either due the number of features etc.. So your strategy should be; 1)for over 100(0) iterations: subsample both cells and genes by %90, cluster using leiden clustering with default parameters, keep track of which pair of cells are clustered together. 2) from the 100(0) iterations, calculate the frequency of pairwise occurrence, and do a final hierarchical clustering using silhouette as a metric. 3) Using the identified clusters, create a PAGA graph and subsequently create a UMAP using the PAGA initialization. The third step will give you a nice visualization overlapping with your clusters identified from 1-2
This will give you a robust clustering

ChatGPT was used for initial code outline but modified to simplify computational usage e.g. through Arda'a idea of doing a dot product of one-hot encoded versions



In [2]:
import scanpy as sc
import numpy as np
import pandas as pd
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage, dendrogram
import itertools

In [2]:

# ... Load your AnnData object (adata) ...

adata= sc.read_h5ad("adata_filtered_combined_feb2025.h5ad")

In [63]:
#num_total = len(adata.obs_names)
#a= np.array([[0,0]] * num_total) #list of length len(adata.obs_names) with [0,0] at each point
#print(a)

#test1 = np.dot(a, a.T)
#print(test1)
#print(test1.shape)

#masking may be the answer -- use a mask to take the subset and then negate the mask to grab the original co-occurence values?

In [64]:
"""
n_iterations = 1000  # Increased iterations for robustness
pairwise_cooccurrence = a= np.zeros((length, length))
original_list_indices=adata.obs_names.to_list()
#print(pairwise_cooccurrence)
#print(len(adata.obs_names))

for i in range(n_iterations):
    # Subsample cells and genes (90%)
    cells_to_keep = np.random.choice(adata.obs_names, size=int(0.9 * len(adata.obs_names)), replace=False)
    genes_to_keep = np.random.choice(adata.var_names, size=int(0.9 * len(adata.var_names)), replace=False)
    adata_subsampled = adata[cells_to_keep, genes_to_keep].copy()
    print(adata_subsampled.obs['leiden'])

    # Leiden clustering (default parameters)
    sc.pp.neighbors(adata_subsampled)
    sc.tl.leiden(adata_subsampled)

    # Track pairwise co-occurrence -- using one hot encoding!!!
    test = pd.get_dummies(adata_subsampled.obs['leiden'])
    #print(test)
    #print(test.index)
    a = np.dot(test,test.T) #dot product counts number of times cells are paired together
    #a.col_names = 
    #print(a.shape)
    #print(a)
    for j,k in itertools.product(range(len(test.index)), range(len(test.index))):
        if a[j][k] == 0:
             continue     
        cell1 = test.index[j]
        #print(cell1)
        cell2 = test.index[k]
        position_j_orig = original_list_indices.index(cell1)
        #print(position_j_orig)
        position_k_orig = original_list_indices.index(cell2)
        #print(position_k_orig)
        pairwise_coocurrence_val = pairwise_cooccurrence[position_j_orig][position_k_orig]
        pairwise_cooccurrence[position_j_orig][position_k_orig] = pairwise_coocurrence_val + a[j][k]
        #print(j)
        #print(k)
    
    print("completed run: " + str(i))
"""              

'\nn_iterations = 1000  # Increased iterations for robustness\npairwise_cooccurrence = a= np.zeros((length, length))\noriginal_list_indices=adata.obs_names.to_list()\n#print(pairwise_cooccurrence)\n#print(len(adata.obs_names))\n\nfor i in range(n_iterations):\n    # Subsample cells and genes (90%)\n    cells_to_keep = np.random.choice(adata.obs_names, size=int(0.9 * len(adata.obs_names)), replace=False)\n    genes_to_keep = np.random.choice(adata.var_names, size=int(0.9 * len(adata.var_names)), replace=False)\n    adata_subsampled = adata[cells_to_keep, genes_to_keep].copy()\n    print(adata_subsampled.obs[\'leiden\'])\n\n    # Leiden clustering (default parameters)\n    sc.pp.neighbors(adata_subsampled)\n    sc.tl.leiden(adata_subsampled)\n\n    # Track pairwise co-occurrence -- using one hot encoding!!!\n    test = pd.get_dummies(adata_subsampled.obs[\'leiden\'])\n    #print(test)\n    #print(test.index)\n    a = np.dot(test,test.T) #dot product counts number of times cells are paire

too computationally expensive --> ideas from Arda:

--when take random subset, keep track of indices chosen so can take leiden clusters on subset data back to normal dimensions before continuing

In [ ]:
#1000 iterations completed on cluster using slurm/sbatch commandline scripy. 
#See clustering2.py in /mnt/pan/SOM_CCCC_JGS25/shultesp/scRNA...etc/
#output was saved to csv as noted below

n_iterations = 1000  # Increased iterations for robustness
length = len(adata.obs_names)
pairwise_cooccurrence = np.zeros((length, length))
indices = np.arange(length)
#print(pairwise_cooccurrence)

for i in range(n_iterations):
    # Subsample cells and genes (90%)
    cells_to_keep_indices = np.random.choice(indices, size=int(0.9 * length), replace=False)
    genes_to_keep = np.random.choice(adata.var_names, size=int(0.9 * len(adata.var_names)), replace=False)
    cells_to_keep = adata.obs_names[cells_to_keep_indices]
    adata_subsampled = adata[cells_to_keep, genes_to_keep].copy()

    # Leiden clustering (default parameters)
    sc.pp.neighbors(adata_subsampled)
    sc.tl.leiden(adata_subsampled)
    #cluster_counts =pd.group(adata_subsampled.obs["leiden"])
    total_cluster_num = adata_subsampled.obs["leiden"].nunique()
    #print(cluster_num)
    
    #expand leiden matrix back to normal dimensions and one-hot-encode
    leiden_matrix = np.zeros((length, total_cluster_num))
    for j in range(length):
        if j not in cells_to_keep_indices:
                continue
        cell = adata.obs_names[j]
        cluster = int(adata_subsampled.obs["leiden"][cell])
        leiden_matrix[j][cluster] = 1
    #print(leiden_matrix)
    
    # Track pairwise co-occurrence -- using one hot encoding!!!
    #print(test)
    #print(test.index)
    a = np.dot(leiden_matrix,leiden_matrix.T) #dot product counts number of times cells are paired together across ALL cells
    pairwise_cooccurrence = pairwise_cooccurrence + a
    #print(pairwise_cooccurrence)
    
    print("completed run: " + str(i))
    #break

final_pairwise_df = pd.DataFrame(pairwise_cooccurrence)
final_pairwise_df.columns = adata.obs_names
final_pairwise_df.index = adata.obs_names

final_pairwise_df.to_csv("cell_pair_counts.csv")
                

completed run: 0
completed run: 1
completed run: 2
completed run: 3
completed run: 4
completed run: 5
completed run: 6
completed run: 7
completed run: 8
completed run: 9
completed run: 10
completed run: 11
completed run: 12
completed run: 13
completed run: 14
completed run: 15
completed run: 16
completed run: 17
completed run: 18
completed run: 19
completed run: 20
completed run: 21
completed run: 22
completed run: 23
completed run: 24
completed run: 25
completed run: 26
completed run: 27
completed run: 28
completed run: 29
completed run: 30
completed run: 31
completed run: 32
completed run: 33
completed run: 34
completed run: 35
completed run: 36
completed run: 37
completed run: 38
completed run: 39
completed run: 40
completed run: 41
completed run: 42
completed run: 43
completed run: 44
completed run: 45
completed run: 46
completed run: 47
completed run: 48
completed run: 49
completed run: 50
completed run: 51
completed run: 52
completed run: 53
completed run: 54
completed run: 55
co

In [ ]:
#load clustering output
cell_pair_counts = pd.read_csv("/mnt/pan/SOM_CCCC_JGS25/shultesp/scRNAseq-clustering-fusion2025/cell_pair_counts.csv")
cell_pair_counts

In [49]:
2. Frequency Calculation and Hierarchical Clustering:

# Convert to a distance matrix (1-normalized co-occurrence)
cooccurrence_matrix_normalized = pairwise_cooccurrence.values / n_iterations
distance_matrix = 1 - cooccurrence_matrix_normalized

# Hierarchical clustering using ward linkage (can experiment with other methods)
linkage_matrix = linkage(distance_matrix, method='ward')

#The silhouette score is not suitable as a metric for hierarchical clustering, since it's a metric to asses cluster quality
#You could calculate the silhouette score on the final clusters after hierarchical clustering

#You can determine the number of clusters by looking at the dendrogram (see below)
#or by using some other metric (e.g. calculating the silhouette score for different numbers of clusters and selecting the one with the best score)
dendrogram(linkage_matrix)
plt.show()
#Cut the dendrogram to get a certain number of clusters
cluster_labels = fcluster(linkage_matrix, t=4, criterion='maxclust') #Example: 4 clusters
adata.obs['consensus_cluster'] = cluster_labels


SyntaxError: invalid syntax (696632085.py, line 1)